In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime

#to capture video from camera
video_capture = cv2.VideoCapture(0)

#to load image from images folder
tata_image=face_recognition.load_image_file("FD_Images/Tata.jpeg")
tata_image=face_recognition.load_image_file("FD_Images/Ratan.jpeg")
#to encode images for processing
tata_encoding=face_recognition.face_encodings(tata_image)[0]
messi_image=face_recognition.load_image_file("FD_Images/Messi.jpeg")
messi_encoding=face_recognition.face_encodings(messi_image)[0]
cr7_image=face_recognition.load_image_file("FD_Images/CR7.jpeg")
cr7_encoding=face_recognition.face_encodings(cr7_image)[0]
jeff_image=face_recognition.load_image_file("FD_Images/Jeff.jpeg")
jeff_encoding=face_recognition.face_encodings(jeff_image)[0]
elon_image=face_recognition.load_image_file("FD_Images/Elon.jpeg")
elon_encoding=face_recognition.face_encodings(elon_image)[0]
shakira_image=face_recognition.load_image_file("FD_Images/Shakira.jpeg")
shakira_encoding=face_recognition.face_encodings(shakira_image)[0]
taylor_image=face_recognition.load_image_file("FD_Images/Taylor.jpeg")
taylor_encoding=face_recognition.face_encodings(taylor_image)[0]

#to store encodings
known_face_encodings = [
    tata_encoding,
    messi_encoding,
    cr7_encoding,
    jeff_encoding,
    elon_encoding,
    shakira_encoding,
    taylor_encoding
]

#to store names of images
known_face_names = [
    "Ratan Tata",
    "Messi",
    "Ronaldo",
    "Jeff Bezos",
    "Elon Musk",
    "Shakira",
    "Taylor Swift"
]

students = known_face_names.copy()
face_locations=[]
face_encodings=[]
face_names=[]
#process frames
process_this_frame = True

#to store current date and time in 'now' variable
now = datetime.now()
#to extract date and time
current_date=now.strftime("%Y-%m-%d")

#to open csv file with name of date and time
f=open(current_date+'.csv','w+',newline='')
#to write in csv file
lnwriter=csv.writer(f)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
            if name in known_face_names:
                if name in students:
                    students.remove(name)
                    print(students)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name,current_time])

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()
f.close()